# Task 1

In [1]:
import hashlib

def feistel_encrypt_block(key:bytes, block:bytes, rounds=16):
    left = block[:len(block)//2]
    right = block[len(block)//2:]
    for i in range(rounds):
        f = hashlib.sha256(right + key + bytes(i)).digest()[:len(right)]
        new_right = bytes([a ^ b for a, b in zip(f, left)])
        left, right = right, new_right
    return left + right

def feistel_decrypt_block(key:bytes, block:bytes, rounds=16):
    left = block[:len(block)//2]
    right = block[len(block)//2:]
    for i in range(rounds-1, -1, -1):
        f = hashlib.sha256(left + key + bytes(i)).digest()[:len(right)]
        new_left = bytes([a ^ b for a, b in zip(f, right)])
        right, left = left, new_left
    return left + right

In [2]:
key = (17855).to_bytes(4, byteorder='big')
feistel_encrypt_block(key, b"welldoneyouXXXXXXXXXcangohomenow", rounds=32).hex()

'34cdbe8fd0643c0323066b4f21b6c07ef65e88a4c7e78d1d07737b352352b5a9'

In [3]:
# Crack
ciphertext = bytes.fromhex("34cdbe8fd0643c0323066b4f21b6c07ef65e88a4c7e78d1d07737b352352b5a9")
for i in range(2**16):
    key = i.to_bytes(4, byteorder='big')
    plaintext = feistel_decrypt_block(key, ciphertext, rounds=32)
    if b"XXXXXXXXX" in plaintext:
        print(plaintext.decode())
        break

welldoneyouXXXXXXXXXcangohomenow


# Task 2

In [4]:
# DES: Data Encryption Standard (1977)

from Crypto.Cipher import DES
from Crypto.Util.Padding import pad, unpad # PKCS7 padding (default)
from secrets import token_bytes

# Key must be 8 bytes long
key = b'\x00\x00\x00\x00\x00' + token_bytes(3)
print(f'Key: {key.hex()}')

cipher = DES.new(key, DES.MODE_ECB)
message = b'It is much more convenient to write this message, because I have a lot more space and I can use spaces as well which is always nice.'

# Pad the message to be a multiple of 8 bytes
padded_message = pad(message, DES.block_size)

encrypted_message = cipher.encrypt(padded_message)
print(f'Encrypted: {encrypted_message.hex()}')

# Decrypt the message
decrypted_message = cipher.decrypt(encrypted_message)

# Unpad the decrypted message
unpadded_message = unpad(decrypted_message, DES.block_size)
print(f'Decrypted: {unpadded_message}')

Key: 00000000000f8c72
Encrypted: 2389a74e47f83172dd2e284b4e4882614900fc2b274d3c7aaa1797f32d7731e86e5b0fdfa68c50b60a62de982900fedeb7102fe582f8c004846efd9e18286ddf65170140968ed4c70b0515b4e65e12f04dee2c904c65093a4314d5b890e4b2f5041c559cb1c65d18c7f7064ff4b0c019c8f21e122994139534becd287713615fecca145312b3b9bf
Decrypted: b'It is much more convenient to write this message, because I have a lot more space and I can use spaces as well which is always nice.'


In [ ]:
encrypted = '2389a74e47f83172dd2e284b4e4882614900fc2b274d3c7aaa1797f32d7731e86e5b0fdfa68c50b60a62de982900fedeb7102fe582f8c004846efd9e18286ddf65170140968ed4c70b0515b4e65e12f04dee2c904c65093a4314d5b890e4b2f5041c559cb1c65d18c7f7064ff4b0c019c8f21e122994139534becd287713615fecca145312b3b9bf'

In [9]:
from collections import Counter
import string

def is_english_text(text: bytes) -> float:
    # Convert bytes to string
    text = text.decode('utf-8', errors='ignore').lower()
    
    # Define the frequency of letters in the English language
    english_letter_freq = {
        'a': 8.167, 'b': 1.492, 'c': 2.782, 'd': 4.253, 'e': 12.702, 'f': 2.228, 'g': 2.015, 'h': 6.094,
        'i': 6.966, 'j': 0.153, 'k': 0.772, 'l': 4.025, 'm': 2.406, 'n': 6.749, 'o': 7.507, 'p': 1.929,
        'q': 0.095, 'r': 5.987, 's': 6.327, 't': 9.056, 'u': 2.758, 'v': 0.978, 'w': 2.360, 'x': 0.150,
        'y': 1.974, 'z': 0.074
    }
    
    # Count the frequency of each letter in the text
    text_letter_freq = Counter(c for c in text if c in string.ascii_lowercase)
    # Calculate the total number of letters in the text
    total_letters = sum(text_letter_freq.values())
    
    if total_letters == 0:
        return 0.0
    
    # Calculate the chi-squared statistic
    chi_squared = 0.0
    for letter, expected_freq in english_letter_freq.items():
        observed_freq = text_letter_freq.get(letter, 0) / total_letters * 100
        chi_squared += ((observed_freq - expected_freq) ** 2) / expected_freq
    
    # Return the inverse of chi-squared as a measure of likelihood
    return 1 / chi_squared

# Example usage
text = b'It is much more convenient to write this message, because I have a lot more space and I can use spaces as well which is always nice.'
likelihood = is_english_text(text)
print(f'Likelihood of being English text: {likelihood}')

Likelihood of being English text: 0.03436525989500543


In [13]:
def increment_bytes(byte_data: bytes) -> bytes:
    byte_list = list(byte_data)
    for i in range(len(byte_list) - 1, -1, -1):
        if byte_list[i] < 255:
            byte_list[i] += 1
            break
        else:
            byte_list[i] = 0
    return bytes(byte_list)


assert increment_bytes(b'\x00\x00\x00') == b'\x00\x00\x01'
assert increment_bytes(b'\x00\x10\x00') == b'\x00\x10\x01'
assert increment_bytes(b'\x00\x00\xff') == b'\x00\x01\x00'
assert increment_bytes(b'\x00\xff\xff') == b'\x01\x00\x00'

In [32]:
max_likelihood = 0.0
best_key = None
best_message = None
ciphertext = bytes.fromhex('c73fb0bfb946de6dba96dddef0e84f4a6590db68a7262923383acfb3b4944e7802f3b9137f87a5e87178b006e746c91690907a3cc9e96de0048fbca86972f8afa176290f433b49b5504a249c0791913f11001eea20161f5fefcccffe120e17a2d484416192cd0a6a0baa73f1ee4728ab45df479650a9e8592542515d84ed7a2e814dd0689cce36dd')

# Generate all possible keys
for i in range(2**(3*8)):
    key = i.to_bytes(3, byteorder='big')
    cipher = DES.new(b'\x00\x00\x00\x00\x00' + key, DES.MODE_ECB)
    message = cipher.decrypt(ciphertext)
    likelihood = is_english_text(message)
    if likelihood > max_likelihood:
        max_likelihood = likelihood
        best_key = key
        best_message = message
    if i % 100000 == 0:
        print(f'Key: {key.hex()}, Likelihood: {max_likelihood}, Message: {best_message}')
        
print(f'Best key: {best_key.hex()}')
print(f'Best message: {best_message.decode()}')

Key: 000000, Likelihood: 0.0008471364722782383, Message: b"\x0eI\xc3\x133\x1eNpu\xf5\xa4\xec\xd2\xe5\xd7\xf1\xa1\xdb\xb8|*\xa0H\x1e\xfd\xfc\xef}W\x8f\xefHZ\x8b.\xce\x0b\xaf\xe0\xa4\xdc^Ts\xd1\xf5h\xc0]\x1b-IP\x10\x9b\xc1\xf2\x00+\xd6[\xa3\rL\x1a\x99\xae\xf8\x06\xf8L\rGN\x11Z\xf2\xe9a}S\x07\xc4ru|\xd9\xad\xe3*\xbe([CS\x80\xc2P\xffeB\x02\xcb7rc'\xf1\xa5\x9f4\xbds\xd3}|V\xeaz\xf8=\xd0;\xc2\x13\xb8\xfc\xad\xd9\x19Y\xee'\xd6\xb8\x99"
Key: 0186a0, Likelihood: 0.014087624347623595, Message: b'\xdc\x05p"6\x02w\tL\xa7[\xf3h\x15\x0e\xca\x8b\xb0\xfb\x00(\x97\x9c\xdd\x02;5o\xe7\x8a\x07{\x90\xba\xfc?\xe6T\x00%\x02\xdc\xd4\xfd\x9c\x0c%u\xf5\x81:\x9b\x17\x99"\xf3\xd4\x1e\xd7\x9a\xac\x81T\xe4\xe2\xc0\xc6\xb9CC\xdc\x1epL\xd92\xe1\t\xa7N\x03E\x01\xc8\x9b7G{N\x8fn\xa9\x0bE9e#e\xc50\x90\xf8\x86*\xe6$\xc4\xaa\xe7\xa9\xe5\xa6\xb1\xdcS\x96bf\xac\xce\xa1\xc7\x171"\xb2A\xe06\xb0r\xa2!\x8f5\xb7'
Key: 030d40, Likelihood: 0.014087624347623595, Message: b'\xdc\x05p"6\x02w\tL\xa7[\xf3h\x15\x0e\xca\x8b\xb0\xfb\x00(\

KeyboardInterrupt: 